In [1]:
import os
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np

# Reading samples

In [ ]:
path_to_raw_files = "/path/to/data/"

files = os.listdir(path_to_raw_files)

samples_dict = {}
for file in files:
    file_path = os.path.join(path_to_raw_files, file)
    
    print(f"Reading sample: {file}")
    # Read 10X data
    adata = sc.read_10x_mtx(file_path, var_names="gene_symbols", cache=True)
    
    print(f"Filtering sample: {file}")
    # Equivalent to min.features = 400 in Seurat (genes per cell)
    sc.pp.filter_cells(adata, min_genes=400)
    
    print(f"Renaming cells for sample: {file}")
    # Rename cells with sample prefix
    adata.obs_names = [f"{file}_{cell}" for cell in adata.obs_names]
    adata.obs["sample"] = file  # keep sample info in metadata
    
    print(f"Finished sample: {file}\n")
    
    samples_dict[file] = adata

# Adding QC metrics

In [ ]:
for key, adata in samples_dict.items():
    sc.pp.calculate_qc_metrics(
        adata, 
        percent_top=None, 
        log1p=False, 
        inplace=True)

# QC plots

## Violin plots

In [ ]:
for key, adata in samples_dict.items():
    print(f"QC plots for {key}")

    # Violin plots (similar to Seurat VlnPlot)
    sc.pl.violin(
        adata,
        keys=["total_counts", "n_genes", "percent_mito", "percent_ribo"], groupby='sample',
        jitter=0.4,
        multi_panel=True
)

## Scatter plots

In [ ]:
for key, adata in samples_dict.items():
    print(f"QC plots for {key}")

    # Scatter plots (similar to FeatureScatter)
    sc.pl.scatter(adata, x="total_counts", y="n_genes")
    # sc.pl.scatter(adata, x="total_counts", y="percent_mito")
    # sc.pl.scatter(adata, x="total_counts", y="percent_ribo")

In [ ]:
# Unlisting samples
for key, adata in samples_dict.items():
    globals()[key] = adata  

In [ ]:
sample1 = sample1[(sample1.obs["total_counts"] < 7000) & (sample1.obs["n_genes_by_counts"] < 8000)]
sample2 = sample2[(sample2.obs["total_counts"] < 5000) & (sample2.obs["n_genes_by_counts"] < 6000)]
# ..

# Filtering by linear model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

def filter_plot_original(adata, sample_name, threshold=-0.5):
    # log-transform variables
    log_counts = np.log1p(adata.obs["total_counts"])
    log_genes = np.log1p(adata.obs["n_genes_by_counts"])
    
    # linear model: log(genes) ~ log(counts)
    X = sm.add_constant(log_counts)  # add intercept
    model = sm.OLS(log_genes, X).fit()
    
    # residuals
    residuals = model.resid
    
    # identify cells to keep
    to_keep = residuals >= threshold
    
    # plotting
    plt.figure(figsize=(6, 5))
    plt.scatter(log_counts, log_genes, c="grey", s=10, label="All cells")
    plt.plot(log_counts, model.predict(X), color="red", linewidth=2, label="Fit")
    plt.scatter(log_counts[~to_keep], log_genes[~to_keep], 
                c="blue", s=10, label="Filtered")
    plt.xlabel("log(total_counts)")
    plt.ylabel("log(n_genes_by_counts)")
    plt.title(sample_name)
    plt.legend()
    plt.show()
    
    if np.sum(to_keep) == 0:
        print(f"No cells retained after filtering in: {sample_name}")
        return adata  # return unfiltered object
    
    # subset AnnData to retained cells
    adata = adata[to_keep, :].copy()
    return adata

In [ ]:
# Apply to all samples
for key in samples_dict:
    samples_dict[key] = filter_plot_original(samples_dict[key], sample_name=key, threshold=-0.5)

# Removing doublets (Scrublet)

In [ ]:
import scrublet as scr
import numpy as np

for key, adata in samples_dict.items():
    print(f"Running Scrublet on {key}...")
    
    # Scrublet expects dense numpy array
    counts_matrix = adata.X.toarray() if not isinstance(adata.X, np.ndarray) else adata.X
    
    scrub = scr.Scrublet(counts_matrix)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()
    
    # Save results in AnnData
    adata.obs["doublet_score"] = doublet_scores
    adata.obs["predicted_doublet"] = predicted_doublets
    
    samples_dict[key] = adata

In [ ]:
# Or 
sc.pp.scrublet(adata, batch_key="sample")

In [ ]:
# Inpsecting doublet score distribution (preferred to set your own threshold rather than the automatic thresholding by scrublet)
for key, adata in samples_dict.items():
    plt.hist(adata.obs["doublet_score"], bins=50)
    plt.title(f"{key} doublet score distribution")
    plt.xlabel("Doublet score")
    plt.ylabel("Cell count")
    plt.show()

## Filter out doublets

In [ ]:
# Define thresholds per sample
thresholds = {
    "sample1": 0.25,
    "sample2": 0.30,
    "sample3": 0.22,
    # add more sample-specific thresholds...
}

In [ ]:
for key, adata in samples_dict.items():
    thr = thresholds.get(key, None)  # None if threshold not defined
    if thr is not None:
        samples_dict[key] = adata[adata.obs["doublet_score"] < thr, :].copy()
        print(f"{key}: kept {samples_dict[key].n_obs} cells after doublet filtering (threshold={thr})")
    else:
        print(f"{key}: no threshold specified, skipping")

# Merging samples

In [ ]:
adata = ad.concat(
    samples_dict.values(),
    label="sample",                # adds a "sample" column in obs
    keys=samples_dict.keys(),      # sample names as labels
    join="outer",                  # keep all genes (outer join)
    index_unique=None              # keep cell barcodes as-is
)

In [ ]:
adata

# Adding metadata information

In [ ]:
# Select columns
adata.obs = adata.obs[['sample', 'n_counts', 'n_genes', 'percent_mito']]

# Add "Condition" column based on "existing column"
adata.obs['Condition'] = adata.obs['orig.ident']

# Replace Condition values based on orig.ident
adata.obs['Condition'] = adata.obs['Condition'].replace(
    {'sample1': 'condition1', 
     'sample2': 'condition2' 
     }
)

## Add a column with a certain value for each observation
adata.obs["column"] = "value"

# Replace based on regex (match) 
adata.obs["column"] = adata.obs["column"].astype(str) # Should be string first
adata.obs.loc[adata.obs["column"].str.match(r"^N"), "column"] = "Normal"
adata.obs.loc[adata.obs["column"].str.match(r"^C"), "column"] = "Cancer"

# Replace based on regex (containing)
adata.obs['age'] = adata.obs['column'].astype(str)
adata.obs.loc[adata.obs["column"].str.contains(r"sample1"), "age"] = "39"

# Add BMI 
adata.obs['BMI'] = adata.obs['sample']
adata.obs['BMI'] = adata.obs['BMI'].replace(to_replace=r'sample1', value='30', regex=True)

# Remove a column
adata.obs = adata.obs.drop(columns=["column"])
## Or
del adata.obs["column"]

# Concatenating
adata.obs['Study_chemistry'] = adata.obs['column1'] + '_' + adata.obs['column2']

# Rename columns
adata.obs = adata.obs.rename(columns={"old_name": "new_name"})

# Separate a column into multiple based on a separator 
adata.obs[["column1", "column2"]] = adata.obs["col"].str.split("_", expand=True)

# Adding column based on a condition
adata.obs["group"] = np.where(adata.obs["BMI"].astype(int) > 25, "overweight", "normal")

# Filtering columns (subset)
# adata = adata[adata.obs["Condition"] == "condition1", :]

# Change a column into categorical
adata.obs['column'] = adata.obs['column'].astype('category') 
# could also use str for string, float for numbers, bool for boolean, 

# show categories of a column 
print(adata.obs['column'].cat.categories)

# Rename a column directly from adata
adata.obs.columns = adata.obs.columns.str.replace('_signature1$', '_signature')

In [ ]:
# Show updated DataFrame
adata.obs.head()

In [ ]:
# Saving merged object
adata.write('adata_merged_filtered.h5ad')

# Data Analysis

In [ ]:
## Reading object
adata = sc.read_h5ad('adata.h5ad')
adata

In [ ]:
# Subsettig clusters to work with (optional)
clusters_to_keep = ["1", "2", "3", "4"]
adata.obs['RNA_snn_res.0.1'] = adata.obs['RNA_snn_res.0.1'].astype(str)
adata = adata[adata.obs['RNA_snn_res.0.1'].isin(clusters_to_keep)].copy()
adata

## Dropping empty levels after subset (if performed)

In [ ]:
# One column
adata.obs['column'] = adata.obs['column'].cat.remove_unused_categories()

In [ ]:
### Multiple columns
for col in adata.obs.select_dtypes(['category']).columns:
    adata.obs[col] = adata.obs[col].cat.remove_unused_categories()

## Normalization

In [ ]:
# Saving raw count data
adata.layers["counts"] = adata.X.copy()

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata, target_sum=1e+06)

# Logarithmize the data
sc.pp.log1p(adata)

## Highly variable gene selection

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample") # flavor='seurat_v3')

# Plotting highly variable genes

In [ ]:
sc.pl.highly_variable_genes(adata)

## Principle component analysis

In [ ]:
sc.tl.pca(adata) #  n_comps=50, svd_solver='arpack'

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True)

You can also plot the principal components to see if there are any potentially undesired features (e.g. batch, QC metrics) driving signifigant variation in this dataset. In this case, there isn’t anything too alarming, but it’s a good idea to explore this.

In [ ]:
sc.pl.pca(
    adata,
    color=["sample", "pct_counts_mt", "pct_counts_mt"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=2,
)

## Nearest neighbor graph construction and visualization

In [ ]:
sc.pp.neighbors(adata)

## UMAP

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(
    adata,
    color="sample", # or ["column1", "column2", "column3"]
    # Setting a smaller point size to get prevent overlap
    size=2,
)

# Data Integration

In [ ]:
# Harmony

In [ ]:
sc.external.pp.harmony_integrate(adata, key='sample', 
                                 basis='X_pca', 
                                 adjusted_basis='X_harmony', 
                                 max_iter_harmony = 100)

### BBKNN

In [ ]:
sc.external.pp.bbknn(adata, 
                     batch_key=var2int, 
                     copy=True)
sc.tl.umap(adata)
sc.pl.umap(adata, color=["sample"])

In [ ]:
import bbknn
bbknn.bbknn(adata, batch_key='sample')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['sample'])
sc.tl.leiden(adata, resolution=0.1)

# Repeat BBKNN integration using leiden as confounder key
bbknn.ridge_regression(adata,
                       batch_key=['samples_study_chemistry'], 
                       confounder_key=['leiden'])
bbknn.bbknn(adata, batch_key='sample')

### MNN

In [ ]:
sc.external.pp.mnn_correct(adata, batch_key = var2int)

### Scanorama

In [ ]:
sc.external.pp.scanorama_integrate(adata, key = var2int)
sc.pp.neighbors(adata, use_rep='X_scanorama')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['sample'])

### scVI

In [ ]:
import scvi
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key=var2int)
scvi_model = scvi.model.SCVI(adata, n_layers=2, n_latent=30)
scvi_model.train()
SCVI_LATENT_KEY = "X_scVI"
adata.obsm[SCVI_LATENT_KEY] = scvi_model.get_latent_representation()

sc.pp.neighbors(adata, use_rep=SCVI_LATENT_KEY)
sc.tl.umap(adata, min_dist=0.3)
sc.pl.umap(
    adata,
    color=["tech"],
    frameon=False,
    ncols=1,
)

# Clustering

`n_iterations`
* Positive values above 2 define the total number of iterations to perform, 
* -1 has the algorithm run until it reaches its optimal clustering. 
* 2 is faster and the default for underlying packages.

In [ ]:
resolutions = np.arange(0.1, 1.1, 0.1)
# Using the igraph implementation and a fixed number of iterations can be significantly faster, 
# especially for larger datasets
for r in resolutions:
    sc.tl.leiden(adata, 
                 resolution=r, 
                 key_added=f'leiden_{r:.1f}')

In [ ]:
# Scanpy tutorial
for res in [0.02, 0.5, 2.0]:
    sc.tl.leiden(
        adata, 
        key_added=f"leiden_res_{res:4.2f}", 
        resolution=res, 
        flavor="igraph"
    )

In [ ]:
## Louvain
for r in resolutions:
    sc.tl.louvain(adata, 
                 resolution=r, 
                 key_added=f'louvain_{r:.1f}')

# Differential Gene Expression as Markers

In [ ]:
chosen_res = ''
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, 
                        groupby=chosen_res, 
                        method="wilcoxon")

In [ ]:
markers_df = sc.get.rank_genes_groups_df(adata, group=None) ## Group None for all clusters
markers_df.head()

In [ ]:
# Exporting markers
markers_df.to_excel('markers.xlsx', index = False)

In [ ]:
# Extracting top n genes
n = 
cluster = ''
markers_df.loc[markers_df["group"] == cluster, "names"].head(n)

In [ ]:
# Top n genes per cluster
n = 
top50 = markers_df.groupby('group').apply(lambda x: x.nlargest(n, 'logfoldchanges')).reset_index(drop=True)
top50

# Plotting top marker genes

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden_res_0.50", standard_scale="var", n_genes=10
)

In [ ]:
marker_genes = ['', '', '', '']
chosen_res = ''
sc.pl.dotplot(adata, 
              marker_genes, 
              groupby=chosen_res, 
              standard_scale="var")

# Cluster annotation (1)

In [ ]:
adata.obs["cell_type"] = adata.obs["leiden_res_0.02"].map(
    {
        "0": "celltype1",
        "1": "celltype2",
        "2": "",
        "3": "",
    }
)

# Cluster annotation (2)

In [ ]:
new_names = {
    '0': 'clus0', 
    '1': 'clus1', 
    '2': 'clus2', 
    '3': 'clus3',
    '4': 'clus4',
    '5': 'clus5',
    '6': 'clus6'}

adata.obs['annotation'] = adata.obs['leiden_0.3'].replace(new_names)

# Gene signature 

In [ ]:
# Reading Excel file
df = pd.read_excel("file.xlsx", sheet_name=0)
df.head()

gene_sets = {col: df[col].dropna().tolist() for col in df.columns}

In [ ]:
for name, genes in gene_sets.items():
    sc.tl.score_genes(
        adata, 
        gene_list=genes, 
        score_name=f"{name}_signature"
    )

## Cluster subset (EXACT one)

In [ ]:
chosen_res = ''
clusters = ['', '']
# Subset data to subset cluster '11'
adata = adata[adata.obs[chosen_res] == clusters].copy()

## Cluster subset (Inclusion of many)

In [ ]:
adata_subset = adata[adata.obs[chosen_res].isin(['4', '6'])]
adata_subset

## Cluster subset (Exclusion of one)

In [ ]:
# Subset data to exclude cluster '11'

adata_subset = adata[adata.obs['leiden_0.3'] != '11'].copy()

## Cluster subset (Exclusion of many)

In [ ]:
adata_subset = adata[~adata.obs[chosen_res].isin(['4', '6'])]
adata_subset